# 01 Modified AMIS (M-AMIS)


In [1]:
! pip install -U pymoo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 615.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 922.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 999.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 923.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for grapheme: filename=grapheme-0.6.0-py3-none-any.whl size=210078 sha256=c9bf19f748bc6f0813edf4b4fd0401f2c7d23f8900e0a5afe486eee196b94f24
  Stored in directory: /root/.cache/pip/wheels/01/e1/49/37e6bde9886439057450c494a79b0bef8bbe897a54aebfc757
Successfully built grapheme


In [2]:
from numpy.core.memmap import uint8
import numpy as np

class AdaptiveWeight:
    def __init__(self, N, Max_iter, F, K):
        self.Ns = np.zeros(( Max_iter+1, N))
        self.As = np.zeros(( Max_iter+1, N))
        self.Is = np.zeros(( Max_iter+1, N))
        self.Iter = 0
        self.Max_iter = Max_iter
        self.N_track = N
        self.pobWeights = np.ones((N,))/N
        self.weights = np.ones((N,))
        self.Ns[0] = np.ones((N,))
        self.As[0] = np.ones((N,))
        self.Is[0] = np.ones((N,))
        self.F = F
        self.K = K
        self.listIndexs = list(np.arange(N, dtype=np.uint8))

    def random_tracks(self, size):
        tracksIds = np.random.choice(self.listIndexs, size, p=self.pobWeights)
        return tracksIds

    def update(self, tracks, objects, bestI):
        self.Iter += 1
        self.Is[self.Iter, bestI] = 1
        for i in range(self.N_track):
            indexs = np.where(tracks==i)[0]
            self.Ns[self.Iter, i] = len(indexs)
            self.As[self.Iter, i] = np.mean(objects[indexs])
            if len(indexs) == 0:
                self.As[self.Iter, i] = 0
            Ii = np.sum(self.Is[:self.Iter, i])

            new_weight = (self.F*self.Ns[self.Iter, i] +
                              (1-self.F)*self.As[self.Iter, i] + self.K*Ii)
            self.weights[i] =  max(0.3*self.weights[i] + 0.7*new_weight, 0)
        total = np.sum(self.weights)
        #print(self.weights, total)
        for i in range(self.N_track):
            self.pobWeights[i] = self.weights[i]/total


In [14]:
from numpy.core.memmap import uint8
import numpy as np

class AdaptiveWeight:
    def __init__(self, N, Max_iter, F, K):
        self.Ns = np.zeros(( Max_iter+1, N))
        self.As = np.zeros(( Max_iter+1, N))
        self.Is = np.zeros(( Max_iter+1, N))
        self.Iter = 0
        self.Max_iter = Max_iter
        self.N_track = N
        self.pobWeights = np.ones((N,))/N
        self.weights = np.ones((N,))
        self.Ns[0] = np.ones((N,))
        self.As[0] = np.ones((N,))
        self.Is[0] = np.ones((N,))
        self.F = F
        self.K = K
        self.listIndexs = list(np.arange(N, dtype=np.uint8))

    def random_tracks(self, size):
        tracksIds = np.random.choice(self.listIndexs, size, p=self.pobWeights)
        return tracksIds

    def update(self, tracks, objects, bestI):
        self.Iter += 1
        self.Is[self.Iter, bestI] = 1
        for i in range(self.N_track):
            indexs = np.where(tracks==i)[0]
            self.Ns[self.Iter, i] = len(indexs)
            self.As[self.Iter, i] = np.mean(objects[indexs])
            if len(indexs) == 0:
                self.As[self.Iter, i] = 0
            Ii = np.sum(self.Is[:self.Iter, i])

            new_weight = (self.F*self.Ns[self.Iter, i] +
                              (1-self.F)*self.As[self.Iter, i] + self.K*Ii)
            self.weights[i] =  max(0.3*self.weights[i] + 0.7*new_weight, 0)
        total = np.sum(self.weights)
        #print(self.weights, total)
        for i in range(self.N_track):
            self.pobWeights[i] = self.weights[i]/total



class MAMIS:
    def __init__(self, problem, pop_size,  max_iter, CR=0.3, CRT=0.5,  F1=0.6, F2=0.4, w=0.9,  gamma=0.4, rho=0.5):
        self.pop_size = pop_size
        self.CR = CR
        self.CRT = CRT
        self.problem = problem
        self.max_iter = max_iter
        self.Xs = None
        self.fitnessXs = None
        self.bestX = None
        self.best2X = None
        self.bestFitness = 0
        self.F1 = F1
        self.F2 = F2
        self.rho = rho
        self.w = w
        self.gamma = gamma
        self.data = np.zeros((3*max_iter, self.problem.n_var))
        self.out = {}
        self.initialize()
        self.funIBs = [self.IB1, self.IB2, self.IB3, self.IB4, self.IB5, self.IB6, self.IB7, self.IB8, self.IB9, self.IB10]
        self.N_blackboxs = len(self.funIBs)
        self.adaptiveWeight = AdaptiveWeight(self.N_blackboxs, max_iter, 0.05, 1)


    def initialize(self):
        self.Xs = np.random.rand( self.pop_size, self.problem.n_var)
        self.fitnessXs = np.zeros((self.pop_size,))
        self.cal_fitness()
        max_index = np.argmax(self.fitnessXs)
        self.bestX = np.copy(self.Xs[max_index])
        self.best2X = np.copy(self.bestX)
        self.bestFitness = self.fitnessXs[max_index]

    def cal_fitness(self):
        for i in range(self.pop_size):
            self.problem._evaluate(self.Xs[i], self.out)
            self.fitnessXs[i] = self.out['F']

    def IB1(self, xs):
        N = len(xs)
        indexs = np.arange(0, self.pop_size, 1).astype(dtype=np.uint8)
        index_r1s = list(np.copy(indexs))
        index_r2s = list(np.copy(indexs))
        np.random.shuffle(index_r1s)
        np.random.shuffle(index_r2s)
        rs = self.Xs[index_r2s[:N]]
        Zs = ((1-self.rho)*rs + self.rho*self.Xs[index_r1s[:N]])*rs
        vmin = np.min(Zs)
        vmax = np.max(Zs)
        if vmin != vmax:
            Zs = (Zs-vmin)/(vmax-vmin)
        return Zs

    def IB2(self, xs):
        N = len(xs)
        indexs = np.arange(0, self.pop_size, 1).astype(dtype=np.uint8)
        index_r1s = list(np.copy(indexs))
        index_r2s = list(np.copy(indexs))
        np.random.shuffle(index_r1s)
        np.random.shuffle(index_r2s)
        rs = self.Xs[index_r2s[:N]]
        Zs = ((1-self.rho)*rs + self.rho*self.Xs[indexs[:N]])*self.Xs[indexs[:N]]
        vmin = np.min(Zs)
        vmax = np.max(Zs)
        if vmin != vmax:
            Zs = (Zs-vmin)/(vmax-vmin)
        return Zs

    def IB3(self, xs):
        N = len(xs)
        indexs = np.arange(0, self.pop_size, 1).astype(dtype=np.uint8)
        index_r1s = list(np.copy(indexs))
        index_r2s = list(np.copy(indexs))
        index_r3s = list(np.copy(indexs))
        np.random.shuffle(index_r1s)
        np.random.shuffle(index_r2s)
        np.random.shuffle(index_r3s)
        Zs = self.Xs[index_r1s[:N]] + self.rho*(self.Xs[index_r2s[:N]] - self.Xs[index_r3s[:N]])

        vmin = np.min(Zs)
        vmax = np.max(Zs)
        if vmin != vmax:
            Zs = (Zs-vmin)/(vmax-vmin)
        return Zs


    def IB4(self, xs):
        N = len(xs)
        indexs = np.arange(0, self.pop_size, 1).astype(dtype=np.uint8)
        index_r1s = list(np.copy(indexs))
        index_r2s = list(np.copy(indexs))
        index_r3s = list(np.copy(indexs))
        np.random.shuffle(index_r1s)
        np.random.shuffle(index_r2s)
        np.random.shuffle(index_r3s)
        Zs = self.bestX + self.rho*(self.Xs[index_r2s[:N]] - self.Xs[index_r3s[:N]])

        vmin = np.min(Zs)
        vmax = np.max(Zs)
        if vmin != vmax:
            Zs = (Zs-vmin)/(vmax-vmin)
        return Zs


    def IB5(self, xs):
        N = len(xs)
        indexs = np.arange(0, self.pop_size, 1).astype(dtype=np.uint8)
        index_r1s = list(np.copy(indexs))
        index_r2s = list(np.copy(indexs))
        index_r3s = list(np.copy(indexs))
        np.random.shuffle(index_r1s)
        np.random.shuffle(index_r2s)
        np.random.shuffle(index_r3s)
        Zs = self.rho*self.bestX + (1-self.rho)*self.Xs[index_r1s[:N]]  + self.gamma* (self.Xs[index_r2s[:N]] - self.Xs[index_r3s[:N]])

        vmin = np.min(Zs)
        vmax = np.max(Zs)
        if vmin != vmax:
            Zs = (Zs-vmin)/(vmax-vmin)
        return Zs

    def IB6(self, xs):
        N = len(xs)
        indexs = np.arange(0, self.pop_size, 1).astype(dtype=np.uint8)
        index_r1s = list(np.copy(indexs))
        np.random.shuffle(index_r1s)
        rs = np.random.rand(xs.shape[0], xs.shape[1])
        Zs = self.w*xs + rs* self.F1* (xs - self.Xs[index_r1s[:N]]) + rs* self.F2* (self.bestX - xs)

        vmin = np.min(Zs)
        vmax = np.max(Zs)
        if vmin != vmax:
            Zs = (Zs-vmin)/(vmax-vmin)
        return Zs

    def IB7(self, xs):
        N = len(xs)
        indexs = np.arange(0, self.pop_size, 1).astype(dtype=np.uint8)
        index_r1s = list(np.copy(indexs))
        index_r2s = list(np.copy(indexs))
        np.random.shuffle(index_r1s)
        np.random.shuffle(index_r2s)
        rs = np.random.rand(xs.shape[0], xs.shape[1])
        Zs = self.w*xs + rs* self.F1* (xs - self.Xs[index_r1s[:N]]) + rs* self.F2* (self.bestX - self.Xs[index_r2s[:N]])

        vmin = np.min(Zs)
        vmax = np.max(Zs)
        if vmin != vmax:
            Zs = (Zs-vmin)/(vmax-vmin)
        return Zs

    def IB8(self, xs):
        N = len(xs)
        rs = np.random.rand(xs.shape[0], xs.shape[1])
        cross_select = np.random.choice(a=[0, 1], size=xs.shape, p=[self.CRT, 1-self.CRT])
        Zs = (1-cross_select)*xs + cross_select*rs
        return Zs

    def IB9(self, xs):
        N = len(xs)
        indexs = np.arange(0, self.pop_size, 1).astype(dtype=np.uint8)
        index_r1s = list(np.copy(indexs))
        np.random.shuffle(index_r1s)
        cross_select = np.random.choice(a=[0, 1], size=xs.shape, p=[self.CRT, 1-self.CRT])
        Zs =  (1 - cross_select)*xs +   cross_select*self.Xs[index_r1s[:N]]
        return Zs


    def IB10(self, xs):
        N = len(xs)
        rs = np.random.rand(xs.shape[0], xs.shape[1])
        cross_select = np.random.choice(a=[0, 1], size=xs.shape, p=[self.CRT, 1-self.CRT])
        Zs = rs
        return Zs


    def select_blocks(self):
        self.Vs = np.copy(self.Xs)
        for i in range(self.N_blackboxs):
            indexI = list(np.where(self.currentTracks == i)[0])
            if len(indexI) == 0:
                continue
            self.Vs[indexI] = self.funIBs[i](self.Xs[indexI])




    def recombination(self):
        cross_select = np.random.choice(a=[0, 1], size=self.Xs.shape, p=[self.CR, 1-self.CR])
        self.Us = (1-cross_select)*self.Xs + cross_select*self.Vs

    def accept_all(self):
        fitnessUs = np.zeros((self.pop_size,))
        select_mask = np.zeros((self.pop_size,))
        for i in range(self.pop_size):
            self.problem._evaluate(self.Us[i], self.out)
            fitnessUs[i] = self.out['F']
            self.fitnessXs[i] = fitnessUs[i]
            self.Xs[i] = np.copy(self.Us[i])
            if fitnessUs[i] > self.bestFitness:
                self.bestFitness = fitnessUs[i]
                self.best2X = np.copy(self.bestX)
                self.bestX = np.copy(self.Us[i])

    def selection(self):
        fitnessUs = np.zeros((self.pop_size,))
        select_mask = np.zeros((self.pop_size,))
        for i in range(self.pop_size):
            self.problem._evaluate(self.Us[i], self.out)
            fitnessUs[i] = self.out['F']
            delta = fitnessUs[i] - self.fitnessXs[i]
            if (delta < 0 or
                np.random.rand() + 5 < np.exp(-delta*100) or
                np.random.rand()  < 0.00001):
                self.fitnessXs[i] = fitnessUs[i]
                self.Xs[i] = np.copy(self.Us[i])
                if fitnessUs[i] < self.bestFitness:
                    self.bestFitness = fitnessUs[i]
                    self.best2X = np.copy(self.bestX)
                    self.bestX = np.copy(self.Us[i])



    def iterate(self):
        for it in range(self.max_iter):
            self.currentTracks = self.adaptiveWeight.random_tracks(self.pop_size)
            self.select_blocks()
            arg_max = np.argmax(self.fitnessXs)
            self.adaptiveWeight.update(self.currentTracks, self.fitnessXs, self.currentTracks[arg_max])
            self.recombination()
            self.selection()

            idx= np.argsort(self.bestX)

            print(it, round(np.mean(self.fitnessXs), 4), round(self.bestFitness, 4), self.adaptiveWeight.pobWeights)
            arg_maxs = np.argsort(self.fitnessXs)
            self.data[it*3:3*(it+1)] = np.copy(self.Xs[arg_maxs[:3]])


from pymoo.algorithms.soo.nonconvex.brkga import BRKGA
from sklearn.metrics import accuracy_score
from pymoo.algorithms.soo.nonconvex.de import DE
import numpy as np
from pymoo.core.problem import ElementwiseProblem
import math

class WC_EnsembleProblem(ElementwiseProblem):

    def __init__(self,preds, ys_c):
        self.N = len(preds)
        self.preds = preds
        self.ys_c = ys_c
        super().__init__(n_var=self.N, n_obj=1, n_constr=0, xl=0, xu=1)



    def _evaluate(self, xs, out, *args, **kwargs):
        w = (xs )/np.sum(xs)
        ypred_en = self.preds[0]*w[0]
        for i in range(1, len( self.preds)):
            ypred_en += (self.preds[i]*w[i])

        indexs = np.where(ypred_en>1)
        ypred_en[indexs] = 1

        ypred_max_en = np.argmax(ypred_en, axis=1)
        d = ypred_en - self.ys_c


        f = 1 - accuracy_score(ys, ypred_max_en)

        out["hash"] = hash(str(w))
        out["F"] = f
        out["pheno"] = {"weights": w}






In [16]:
from pymoo.core.callback import Callback
from pymoo.optimize import minimize
import numpy as np
from pymoo.core.problem import ElementwiseProblem
import math

class SampleProblem(ElementwiseProblem):

    def __init__(self, N):
        self.N = N
        super().__init__(n_var=self.N, n_obj=1, n_constr=0, xl=0, xu=1)

    def _function_cost(self, x):
        x = x[0]
        return x*x - 0.5

    def _evaluate(self, x, out, *args, **kwargs):

        f = self._function_cost(x)


        out["hash"] = hash(str(x))
        out["F"] = f
        #out["pheno"] = {"values": solution}

problem = SampleProblem(1)
np.random.seed(0)

algorithm = MAMIS(problem,
    pop_size=20,
    CR=0.3,
    max_iter = 100,
    #dither="vector",
    #jitter=False
)
algorithm.iterate()

0 -0.1589 -0.5 [0.06900042 0.08415141 0.10942038 0.09775429 0.09356591 0.13012437
 0.1067401  0.09909359 0.09775429 0.11239524]
1 -0.2632 -0.5 [0.11054152 0.06389684 0.08477545 0.11546605 0.10227403 0.14525875
 0.09936969 0.09952177 0.08702655 0.09186932]
2 -0.2961 -0.5 [0.07202521 0.0867886  0.09304695 0.14145934 0.11936853 0.17201585
 0.0878514  0.06872205 0.07549138 0.08323067]
3 -0.3139 -0.5 [0.05648479 0.07198696 0.084712   0.14223954 0.1510666  0.16824833
 0.09724417 0.05555958 0.08429393 0.0881641 ]
4 -0.3844 -0.5 [0.05130372 0.07294225 0.10495274 0.14500479 0.15765349 0.14754572
 0.078196   0.05116286 0.06455484 0.1266836 ]
5 -0.423 -0.5 [0.04606191 0.06215168 0.12670229 0.13541159 0.1373811  0.14431918
 0.06706757 0.0694826  0.07314686 0.1382752 ]
6 -0.4432 -0.5 [0.05806566 0.07013669 0.12917228 0.13414309 0.12766616 0.17876435
 0.05184625 0.05258634 0.0574356  0.14018358]
7 -0.4432 -0.5 [0.06430796 0.04953185 0.11652171 0.13358608 0.11613959 0.17366293
 0.06259177 0.05518277 

#02 Average Ensemble

In [18]:
def avg_ensemble(model_name, preds, ys):
  ypred1, ypred2, ypred3, ypred4, ypred5 = preds
  print('Report :',datetime.datetime.now())
  print('\n')
  ypred_max_m1 = np.argmax(ypred1, axis=1)
  print('Model 1 : ', model_name, ' has Accuracy ',accuracy_score(ys, ypred_max_m1))
  print('Model 1 : ', model_name, ' has AUC ',roc_auc_score(ys, ypred1, multi_class='ovr'))
  print('Model 1 : ', model_name, ' has F1 ',f1_score(ys, ypred_max_m1,average='weighted'))
  print('\n')
  ypred_max_m2 = np.argmax(ypred2, axis=1)
  print('Model 2 : ', model_name, ' has Accuracy ',accuracy_score(ys, ypred_max_m2))
  print('Model 2 : ', model_name, ' has AUC ',roc_auc_score(ys, ypred2, multi_class='ovr'))
  print('Model 2 : ', model_name, ' has F1 ',f1_score(ys, ypred_max_m2,average='weighted'))
  print('\n')
  ypred_max_m3 = np.argmax(ypred3, axis=1)
  print('Model 3 : ', model_name, ' has Accuracy ',accuracy_score(ys, ypred_max_m3))
  print('Model 3 : ', model_name, ' has AUC ',roc_auc_score(ys, ypred3, multi_class='ovr'))
  print('Model 3 : ', model_name, ' has F1 ',f1_score(ys, ypred_max_m3,average='weighted'))
  print('\n')
  ypred_max_m4 = np.argmax(ypred4, axis=1)
  print('Model 4 : ', model_name, ' has Accuracy ',accuracy_score(ys, ypred_max_m4))
  print('Model 4 : ', model_name, ' has AUC ',roc_auc_score(ys, ypred4, multi_class='ovr'))
  print('Model 4 : ', model_name, ' has F1 ',f1_score(ys, ypred_max_m4,average='weighted'))
  print('\n')
  ypred_max_m5 = np.argmax(ypred5, axis=1)
  print('Model 5 : ', model_name, ' has Accuracy ',accuracy_score(ys, ypred_max_m5))
  print('Model 5 : ', model_name, ' has AUC ',roc_auc_score(ys, ypred5, multi_class='ovr'))
  print('Model 5 : ', model_name, ' has F1 ',f1_score(ys, ypred_max_m5,average='weighted'))
  print('\n')

  ypred_en = (ypred1 + ypred2 + ypred3 + ypred4 + ypred5)/5
  ypred_max_en = np.argmax(ypred_en, axis=1)
  print('Ensemble Model : ', model_name, ' has Accuracy ',accuracy_score(ys, ypred_max_en))
  print('Ensemble Model : ', model_name, ' has AUC ',roc_auc_score(ys, ypred_en, multi_class='ovr'))
  print('Ensemble Model : ', model_name, ' has F1 ',f1_score(ys, ypred_max_en,average='weighted'))

In [ ]:
from keras import models
from keras import backend as k

if k.backend() == 'tensorflow':
  print('Clear old session')
  k.clear_session()

model_path = "./Model"
model_name = "MobileNetV2"

Xs = np.load("./Dataset/Xs.npy")
ys = np.load("./Dataset/Xs.npy")

print('Report :',datetime.datetime.now())
print('\n Load Model ', model_name)
model1 = models.load_model(model_path+model_name+"_1.tf")
model2 = models.load_model(model_path+model_name+"_2.tf")
model3 = models.load_model(model_path+model_name+"_3.tf")
model4 = models.load_model(model_path+model_name+"_4.tf")
model5 = models.load_model(model_path+model_name+"_5.tf")

print('\n Predict')
start = datetime.datetime.now()
ypred1 = model1.predict(Xs)
ypred2 = model2.predict(Xs)
ypred3 = model3.predict(Xs)
ypred4 = model4.predict(Xs)
ypred5 = model5.predict(Xs)

ypreds = [ypred1, ypred2, ypred3, ypred4,ypred5]

In [ ]:
avg_ensemble(model_name, ypreds, ys)

#03 Weight Ensemble Problem

In [17]:
from pymoo.algorithms.soo.nonconvex.brkga import BRKGA
from pymoo.algorithms.soo.nonconvex.de import DE
import numpy as np
from pymoo.core.problem import ElementwiseProblem
import math

class WC_EnsembleProblem(ElementwiseProblem):

    def __init__(self,preds, ys_c):
        self.N = len(preds)
        self.preds = preds
        self.ys_c = ys_c
        super().__init__(n_var=self.N, n_obj=1, n_constr=0, xl=0, xu=1)



    def _evaluate(self, xs, out, *args, **kwargs):
        w = (xs )/np.sum(xs)
        ypred_en = self.preds[0]*w[0]
        for i in range(1, len( self.preds)):
            ypred_en += (self.preds[i]*w[i])

        indexs = np.where(ypred_en>1)
        ypred_en[indexs] = 1

        ypred_max_en = np.argmax(ypred_en, axis=1)
        d = ypred_en - self.ys_c


        f = 1 - accuracy_score(ys, ypred_max_en)

        out["hash"] = hash(str(w))
        out["F"] = f
        out["pheno"] = {"weights": w}

In [ ]:

from pymoo.core.callback import Callback
from pymoo.optimize import minimize
from tensorflow.keras.utils import to_categorical

y_c = to_categorical(ys, 5)
problem = WC_EnsembleProblem(ypreds, y_c)
np.random.seed(0)

algorithm = MAMIS(problem,
    pop_size=50,
    CR=0.3,
    max_iter = 3000,
    #dither="vector",
    #jitter=False
)
algorithm.iterate()